In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import shutil
import random
random.seed(42)
import tqdm

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import cv2
from sklearn import preprocessing

In [2]:
train_data=pd.read_csv("../input/petfinder-pawpularity-score/train.csv").to_numpy()
test_data=pd.read_csv("../input/petfinder-pawpularity-score/test.csv").to_numpy()

train_path="../input/petfinder-pawpularity-score/train"
test_path="../input/petfinder-pawpularity-score/test"

dimensions=(512,512,3)

In [3]:
import tensorflow as tf

In [4]:
import keras
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory, list_IDs, n_channels=3, batch_size=8, dim=dimensions, shuffle=True):
        'Initialization'
        self.dim = dim
        self.directory = directory
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        
        X = np.empty((self.batch_size, *self.dim))
        y = []
            
        for batch_number, id_temp_entry in enumerate(list_IDs_temp):
            path = self.directory + '/' + id_temp_entry[0] + '.jpg'
                
            image = cv2.imread(path)# cv.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (self.dim[0],self.dim[1]))
            image = image.reshape(dimensions)
            
            X[batch_number] = image
            
            if len(id_temp_entry) >= 14:
                y.append(id_temp_entry[13])
            
        y = np.array(y)
        if len(y) == 0:
            return X, []
        
        
        return X,  y
                     
    def __getitem__(self, index):
            
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X, y = self.__data_generation(list_IDs_temp)
        
        if len(y) == 0:
            return X

        return X, y

In [5]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import models
from tensorflow.keras import layers

#https://qiita.com/kuroneko-hornet/items/7737b71c3854c06fcb49
def build_model():
    conv_base=inception_v3.InceptionV3(weights=None,
                  include_top=False,
                  input_shape=dimensions# resnetを使う場合、channelを3にしないといけない
                      )

    model=models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(1))

    return model

In [6]:
model=build_model()

2021-11-21 13:11:18.804498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 13:11:18.911726: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 13:11:18.912424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-21 13:11:18.913591: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.MeanSquaredError(),metrics=["mse","mae"])

In [8]:
training_generator=DataGenerator(train_path, train_data, batch_size=32)
validation_generator=DataGenerator(test_path, test_data, batch_size=1)

model.fit(training_generator,
          validation_data=validation_generator, epochs=5)
model.summary()
results=model.predict(validation_generator)

2021-11-21 13:11:23.989845: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5


2021-11-21 13:11:31.571800: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


309/309 [==============================] - 275s 840ms/step - loss: 5378.5449 - mse: 5378.5449 - mae: 40.8251 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/5
309/309 [==============================] - 226s 729ms/step - loss: 1851.6058 - mse: 1851.6058 - mae: 37.7880 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 3/5
309/309 [==============================] - 226s 730ms/step - loss: 1835.3081 - mse: 1835.3081 - mae: 37.5585 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 4/5
309/309 [==============================] - 226s 731ms/step - loss: 1815.3839 - mse: 1815.3839 - mae: 37.2985 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 5/5
309/309 [==============================] - 227s 733ms/step - loss: 1794.5920 - mse: 1794.5920 - mae: 37.0158 - val_loss: 0.0000e+00 - val_mse: 0.0000e+00 - val_mae: 0.0000e+00
Model: "sequential"
____________________________________________________________

In [9]:
submission = []

for index, result in enumerate(results):
    submission.append([test_data[index][0], float(result[0])])
    
    
df = pd.DataFrame(submission, columns=['Id', 'Pawpularity'])
df = df.sort_values('Id')
df.to_csv('submission.csv', index=False)